This netbook retrieves price data from the Yahoo! Finance API and quarterly earnings releases from the Alpha Vantage API to build a DataFrame that may be further used to train Machine Learning models to forecast price change after quarterly earnings releases:

**1. Import all necessary libraries:**

In [1]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as datetime

**2. Create a DataFrame with all OHLC price data of the stock:**

In [2]:
IBM = yf.download('IBM', period='max')

[*********************100%***********************]  1 of 1 completed


We add a new column, the price change between the day before the earnings call and the day afterwards, as the earnings may be before the open of the day or after the close:

In [3]:
IBM['Return_2D'] = (IBM['Open'].shift(-1) - IBM['Close'].shift(1))/IBM['Close'].shift(1)*100

**3. Create a DataFrame with the earnings calendar:**

In [4]:
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=IBM&apikey=demo'
r = requests.get(url)
data_IBM = r.json()

In [5]:
quarterly_earnings = []
for i in range(len(data_IBM['quarterlyEarnings'])):
    quarterly_earnings.append(data_IBM['quarterlyEarnings'][i])
quarterly_earnings = pd.DataFrame(quarterly_earnings)
quarterly_earnings.head()

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2023-03-31,2023-04-19,1.36,1.26,0.1,7.9365
1,2022-12-31,2023-01-25,3.6,3.6,0,0
2,2022-09-30,2022-10-19,1.81,1.77,0.04,2.2599
3,2022-06-30,2022-07-18,2.31,2.27,0.04,1.7621
4,2022-03-31,2022-04-19,1.4,1.38,0.02,1.4493


In [6]:
quarterly_earnings['reportedDate'] = pd.to_datetime(quarterly_earnings['reportedDate'])

In [7]:
quarterly_earnings.index=quarterly_earnings['reportedDate']

In [8]:
quarterly_earnings.head()

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
reportedDate,,,,,,
2023-04-19,2023-03-31,2023-04-19,1.36,1.26,0.1,7.9365
2023-01-25,2022-12-31,2023-01-25,3.6,3.6,0,0
2022-10-19,2022-09-30,2022-10-19,1.81,1.77,0.04,2.2599
2022-07-18,2022-06-30,2022-07-18,2.31,2.27,0.04,1.7621
2022-04-19,2022-03-31,2022-04-19,1.4,1.38,0.02,1.4493


**4. Join both DataFrames:**

In [9]:
IBM = IBM.join(quarterly_earnings, lsuffix='_left', rsuffix='_right')

In [10]:
IBM.head()

,Open,High,Low,Close,Adj Close,Volume,Return_2D,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
Date,,,,,,,,,,,,,
1962-01-02,7.374124,7.374124,7.291268,7.291268,1.591517,407940,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1962-01-03,7.291268,7.355003,7.291268,7.355003,1.605427,305955,0.874128,NaN,NaT,NaN,NaN,NaN,NaN
1962-01-04,7.355003,7.355003,7.278521,7.281708,1.589429,274575,-1.126509,NaN,NaT,NaN,NaN,NaN,NaN
1962-01-05,7.272148,7.272148,7.125558,7.138305,1.558128,384405,-2.056893,NaN,NaT,NaN,NaN,NaN,NaN
1962-01-08,7.131931,7.131931,6.947100,7.004461,1.528912,572685,-1.428580,NaN,NaT,NaN,NaN,NaN,NaN


We then drop all empty rows to keep only the days with earning releases.

In [11]:
IBM = IBM.dropna()

In [12]:
IBM.head()

,Open,High,Low,Close,Adj Close,Volume,Return_2D,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
Date,,,,,,,,,,,,,
1996-04-17,27.903919,28.710564,24.707218,25.155354,13.747612,68195434,-8.549784,1996-03-31,1996-04-17,0.55,0.6,-0.05,-8.3333
1996-07-25,22.944551,24.796844,22.705545,24.766970,13.580250,37718342,12.397823,1996-06-30,1996-07-25,0.63,0.61,0.02,3.2787
1996-10-21,30.592733,32.355404,30.293978,31.100620,17.107744,34870711,0.193237,1996-09-30,1996-10-21,0.61,0.61,0,0
1997-01-21,39.764580,40.660851,39.764580,40.152966,22.146753,15682887,-4.790425,1996-12-31,1997-01-21,0.98,0.99,-0.01,-1.0101
1997-04-23,33.460804,34.357075,33.221798,34.028442,18.812975,16389146,7.499998,1997-03-31,1997-04-23,0.59,0.58,0.01,1.7241


In [15]:
print(round(abs(IBM['Return_2D']).mean(), 2))

4.29


As we can see, IBM shows an average 4.29% price movement during earning releases. The prediction of this heavy movement is a topic for further research using Machine Learning models.